## Preprocessing data_2

### Loading the Weather data

In [ ]:
import pandas as pd
import numpy as np
import glob
import json

In [2]:
airports = ["ATL", "CLT", "DEN", "DFW", "EWR", "IAH", "JFK", "LAS", "LAX", "MCO", "MIA", "ORD", "PHX", "SEA", "SFO"]
Weather_columns =['windspeedKmph', 'winddirDegree',  'weatherCode', 'precipMM', 'visibility', 'pressure', 
                   'cloudcover', 'DewPointF', 'WindGustKmph', 'tempF',  'WindChillF', 'humidity', 'time']

In [3]:
weather_files_2016 = glob.glob('Data\Weather\\***\\2016-***.json')  # Reading all json file paths from 2016
weather_2016 = []

# Starting a loop which iterates through the paths of from the path list
for file in weather_files_2016:
    with open(file) as f:
        temp = json.load(f)
        for series in temp['data']['weather']: # Iterating through the file and the weather module
            df_new = pd.DataFrame(series['hourly'],columns=Weather_columns)
            df_new['date'] = series['date']          # Adding the date column
            for name in airports:                        # Adding the Airport code based on the file name
                if name in file.split('\\'):
                    df_new['airport'] = name
            weather_2016 = pd.DataFrame(weather_2016.append(df_new))  # Appending the dataframe

In [4]:
weather_2016.head()

,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,time,date,airport
0,15,319,122,0.0,10,1026,86,33,23,40,34,76,0,2016-01-02,ATL
1,16,320,122,0.0,10,1026,81,33,23,39,33,78,100,2016-01-02,ATL
2,16,321,116,0.0,10,1026,76,33,23,38,32,80,200,2016-01-02,ATL
3,16,322,116,0.0,10,1026,71,33,23,38,31,83,300,2016-01-02,ATL
4,16,319,116,0.0,10,1026,79,32,23,37,30,83,400,2016-01-02,ATL


In [5]:
weather_files_2017 = glob.glob('Data\Weather\\***\\2017-***.json')  # Reading all json file paths from 2017
weather_2017 = []

# Starting a loop which iterates through the paths of from the path list
for file in weather_files_2017:
    with open(file) as f:
        temp = json.load(f)
        for series in temp['data']['weather']: # Iterating through the file and the weather module
            df_new = pd.DataFrame(series['hourly'],columns=Weather_columns)
            df_new['date'] = series['date']          # Adding the date column
            for name in airports:                        # Adding the Airport code based on the file name
                if name in file.split('\\'):
                    df_new['airport'] = name
            weather_2017 = pd.DataFrame(weather_2017.append(df_new))  # Appending the dataframe
weather_2017.head()

,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,time,date,airport
0,8,100,353,6.1,7,1021,100,50,16,51,49,97,0,2017-01-02,ATL
1,8,99,353,5.1,7,1020,100,50,16,51,50,97,100,2017-01-02,ATL
2,7,99,356,4.1,8,1020,100,51,15,52,50,98,200,2017-01-02,ATL
3,7,99,356,3.1,9,1020,100,51,14,52,51,98,300,2017-01-02,ATL
4,6,92,356,4.1,8,1020,100,52,13,52,52,98,400,2017-01-02,ATL


In [6]:
# Using `pd.concat` to merge both the weather datasets.
final_weather_dataset = pd.concat([weather_2016,weather_2017])
final_weather_dataset.head()

,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,time,date,airport
0,15,319,122,0.0,10,1026,86,33,23,40,34,76,0,2016-01-02,ATL
1,16,320,122,0.0,10,1026,81,33,23,39,33,78,100,2016-01-02,ATL
2,16,321,116,0.0,10,1026,76,33,23,38,32,80,200,2016-01-02,ATL
3,16,322,116,0.0,10,1026,71,33,23,38,31,83,300,2016-01-02,ATL
4,16,319,116,0.0,10,1026,79,32,23,37,30,83,400,2016-01-02,ATL


In [12]:
final_weather_dataset.to_csv("Data\Final_Weather_Data.csv")

In [13]:
final_weather_dataset = pd.read_csv("Data\Final_Weather_Data.csv")

## Merging flight and weather data

### Creating a new time column in flight dataset

In [14]:
df_flight = pd.read_csv('Data\Final_Flight_Data.csv')

In [15]:
# Creating an empty list that will act as the series we will add to the dataframe
time_list = []
# Iterating through the flight data series CRSDepTime
for i in df_flight["CRSDepTime"]:
    # Converting time to int for logical comparision
    i = int(i)
    # Dividing by 100 and saving to new variable
    x = i%100
    # If it is less than 30 we just subtract it from the time
    if(x<30) :
        i = i - x
    # If it is greater than 30 we subtract it from 100  and add
    elif(x>=30) :
        i = i+(100 - x)
    # If it is 2400 we just make it 0
    if(i == 2400):
        i = 0
    # Appending to the series
    time_list.append(i)
df_flight['Time_new'] = time_list

In [16]:
# Merging the data on the airport departure time and date
df = pd.merge(df_flight, final_weather_dataset,
              how = 'inner',
              left_on = ['Origin','Time_new', 'FlightDate'],
              right_on = ['airport','time', 'date'])

In [17]:
df.to_csv('Data/Final_data.csv')

In [18]:
# Reading the final dataframe file
import pandas as pd
df = pd.read_csv('Data/Final_data.csv', index_col = 0)

C:\Users\charu\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [20]:
# checking if all features have same number of rows
df.head()

,Unnamed: 0_x,ArrDel15,ArrDelayMinutes,ArrTime,CRSArrTime,CRSDepTime,DayofMonth,DepDel15,DepDelayMinutes,DepTime,...,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,time,date,airport
0,0,0.0,8.0,1610.0,1602,745,1,0.0,0.0,741.0,...,1030,0,23,8,34,29,65,800,2016-01-01,SEA
1,12938,1.0,33.0,1237.0,1204,820,1,0.0,1.0,821.0,...,1030,0,23,8,34,29,65,800,2016-01-01,SEA
2,52237,0.0,0.0,1138.0,1143,759,1,0.0,0.0,754.0,...,1030,0,23,8,34,29,65,800,2016-01-01,SEA
3,52492,0.0,10.0,1401.0,1351,738,1,0.0,2.0,740.0,...,1030,0,23,8,34,29,65,800,2016-01-01,SEA
4,62876,0.0,0.0,1524.0,1527,745,1,0.0,0.0,742.0,...,1030,0,23,8,34,29,65,800,2016-01-01,SEA
